In [166]:
# imports
import pandas as pd

# 2. data handling basics

## Series and DataFrames

the pandas package provides a comfortable representation of **tabular** data in `DataFrame`s.

These are common ways to create `DataFrame`s:

In [167]:
# create df using a dictionary:
df = pd.DataFrame({"name": ["milan", "jürgen"], "age": [24, 32]})

# read a data frame from url (and save it)
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)
df.to_csv(path_or_buf="../data/2/titanic.csv")

# create df by reading a csv file from our memory
df = pd.read_csv("../data/2/titanic.csv")

In [168]:
shape = df.shape  # (n_rows, n_cols)
df.head()  # first 5 rows

,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [169]:
# drop a row, axis 0 = rows
df = df.drop(0, axis=0)  # drop first row
df.head()

# drop 2 a columns, axis 1 = columns
df = df.drop(["Unnamed: 0", "PassengerId"], axis=1)
df.head()

# change row ids
columns = list(df.columns)  # names of columns
columns[-1] = "Harbor"
df.columns = columns

# change index ids
index = df.index
df.index = range(len(index))

## indexing and slicing
`indexing` = accessing single entries (single row, single column)

`slicing` = accessing multiple entries (multiple rows, multiple columns)

In [170]:
# indexing
first_row = df.loc[0, :]
first_column = df.loc[:, df.columns[0]]

# slicing
first_three_rows = df.loc[:2, :]  # 2 included
first_three_columns = df.loc[:, df.columns[:3]]

## masking
masking can be seen as a special case of `indexing` / `slicing`. instead of accessing certain indices, masking filters the `DataFrame` and only returns those elements that fulfill the filter `conditional`s.

In [171]:
# rows where age < 30
df_young = df[df["Age"] < 30]

# rows where age < 30 and sex = female
df_young_female = df[(df["Age"] < 30) & (df["Sex"] == "female")]

# rows where age < 30 or name contains "Miss."
df_young_or_miss = df[(df["Age"] < 30) | (df["Name"].str.contains("Miss."))]

## null values
null values are cells within tabular data that contain blank spaces. elaborate data analysis and machine learning algorithms can't handle such missing values.

In [172]:
# find out number of missing values per column
df.isna().sum()

# drop rows with missing values
df.dropna(axis=0)

# drop columns with missing values
df.dropna(axis=1)

# drop rows with more than 2 missing values
df.dropna(axis=0, thresh=len(df.columns)-2)

# replace missing values with 0
df.fillna(value=0)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Harbor
0,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
1,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,0,S
2,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
3,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,0,S
4,0,3,"Moran, Mr. James",male,0.0,0,0,330877,8.4583,0,Q
...,...,...,...,...,...,...,...,...,...,...,...
885,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,0,S
886,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
887,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,0.0,1,2,W./C. 6607,23.4500,0,S
888,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## appending new columns to a data frame

In [173]:
# append a single column
new_col = pd.Series([0] * len(df), name="Zero")
df["Zero"] = new_col

# create one new column from existing one
df["Age Squared"] = df["Age"]**2

# create one new column with apply
def sex_to_bool(sex: str) -> bool:
    if sex == "female":
        return True
    return False
df["Female"] = df["Sex"].apply(func=sex_to_bool)

# create multiple columns with apply
def sex_to_one_hot(sex: str) -> pd.Series:
    if sex == "female":
        return pd.Series({"F": 1, "M": 0})
    return pd.Series({"F": 0, "M": 1})
df[["F", "M"]] = df["Sex"].apply(func=sex_to_one_hot)

## merging data frames

In [174]:
# create two example data frames
df1 = pd.DataFrame({"key": ["A", "B"], "value": [1, 2]})
df2 = pd.DataFrame({"key": ["B", "C"], "value": [3, 4]})

In [175]:
# inner
pd.merge(left=df1, right=df2, on="key", how="inner")

,key,value_x,value_y
0,B,2,3


In [176]:
# outer
pd.merge(left=df1, right=df2, on="key", how="outer")

,key,value_x,value_y
0,A,1.0,NaN
1,B,2.0,3.0
2,C,NaN,4.0


In [177]:
# left
pd.merge(left=df1, right=df2, on="key", how="left")

,key,value_x,value_y
0,A,1,NaN
1,B,2,3.0


In [178]:
# right
pd.merge(left=df1, right=df2, on="key", how="right")

,key,value_x,value_y
0,B,2.0,3
1,C,NaN,4


## sorting

In [179]:
df = pd.read_csv("../data/2/titanic.csv")

# sort by age
df.sort_values(by="Age", ascending=False)

# sort by priority 1: Age, priority 2: PassengerId
df.sort_values(by=["Age", "PassengerId"], ascending=False)

,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
630,630,631,1,1,"Barkworth, Mr. Algernon Henry Wilson",male,80.0,0,0,27042,30.0000,A23,S
851,851,852,0,3,"Svensson, Mr. Johan",male,74.0,0,0,347060,7.7750,NaN,S
493,493,494,0,1,"Artagaveytia, Mr. Ramon",male,71.0,0,0,PC 17609,49.5042,NaN,C
96,96,97,0,1,"Goldschmidt, Mr. George B",male,71.0,0,0,PC 17754,34.6542,A5,C
116,116,117,0,3,"Connors, Mr. Patrick",male,70.5,0,0,370369,7.7500,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
26,26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
19,19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
17,17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S


## uniques

In [180]:
# number of unique values per column
df.nunique()

# each unique value in column "Age" mapped
# to the number of rows in which in occurs
df["Age"].value_counts()

# all unique values in the column "Age"
df["Age"].unique()

array([22.  , 38.  , 26.  , 35.  ,   nan, 54.  ,  2.  , 27.  , 14.  ,
        4.  , 58.  , 20.  , 39.  , 55.  , 31.  , 34.  , 15.  , 28.  ,
        8.  , 19.  , 40.  , 66.  , 42.  , 21.  , 18.  ,  3.  ,  7.  ,
       49.  , 29.  , 65.  , 28.5 ,  5.  , 11.  , 45.  , 17.  , 32.  ,
       16.  , 25.  ,  0.83, 30.  , 33.  , 23.  , 24.  , 46.  , 59.  ,
       71.  , 37.  , 47.  , 14.5 , 70.5 , 32.5 , 12.  ,  9.  , 36.5 ,
       51.  , 55.5 , 40.5 , 44.  ,  1.  , 61.  , 56.  , 50.  , 36.  ,
       45.5 , 20.5 , 62.  , 41.  , 52.  , 63.  , 23.5 ,  0.92, 43.  ,
       60.  , 10.  , 64.  , 13.  , 48.  ,  0.75, 53.  , 57.  , 80.  ,
       70.  , 24.5 ,  6.  ,  0.67, 30.5 ,  0.42, 34.5 , 74.  ])

## aggregations

In [181]:
# maximum age
df["Age"].max()

# row id of the passenger with maximum age
df["Age"].idxmax()

# standard deviation of the passenger ages
df["Age"].std()

14.526497332334042

## groupby

In [182]:
# mean age grouped by sex
df.groupby("Sex")["Age"].mean()

# group by sex, per group provides
# passenger id of the person with
# minimum fare
df.groupby("Sex")["Fare"].idxmin()

Sex
female    654
male      179
Name: Fare, dtype: int64